In [1]:
import numpy as np
import tensorflow as tf
import queue
import pickle
import zipfile
import collections
import math
import time

from collections import Counter

In [2]:
from train_util import *
from preprocessing import *
from layer import *
from model import *

In [3]:
test_word2vec()

Filling Unigram Table
Testing Model :cbow, loss : Softmax
idx: 200, loss: 2.076458, time: 0.028922, 
idx: 200, loss: 2.081181, time: 0.026928, 
idx: 200, loss: 2.077710, time: 0.024936, 
idx: 200, loss: 2.079751, time: 0.027925, 
idx: 200, loss: 2.070858, time: 0.025932, 
idx: 200, loss: 2.072603, time: 0.026927, 
idx: 200, loss: 2.067445, time: 0.024439, 
idx: 200, loss: 2.062257, time: 0.033911, 
idx: 200, loss: 2.055293, time: 0.025950, 
idx: 200, loss: 2.051205, time: 0.026907, 
Testing Model :cbow, loss : Negative_Sampling
idx: 200, loss: 2.076036, time: 0.024933, 
idx: 200, loss: 2.074896, time: 0.025931, 
idx: 200, loss: 2.073688, time: 0.024958, 
idx: 200, loss: 2.073738, time: 0.025935, 
idx: 200, loss: 2.079159, time: 0.026899, 
idx: 200, loss: 2.073700, time: 0.024934, 
idx: 200, loss: 2.072034, time: 0.025931, 
idx: 200, loss: 2.072498, time: 0.025928, 
idx: 200, loss: 2.069589, time: 0.024934, 
idx: 200, loss: 2.070625, time: 0.028943, 
Testing Model :cbow, loss : Hierarch

1. Preprocessing the data
===

In [4]:
#parameters
vocab_size = 100000
thresholds = 1e-3
embedding_size = 200
context_size = 5
architecture = 'cbow'
use_loss = 'Negative_Sampling'
sample_size = 25
lr = 1e-3
epochs = 10

In [5]:
words = preprocess('text8.zip')

In [6]:
words, word_lst, word_dict, word_reverse_dict, words_index, unigram_dict = total_sampling(file_data=words, subsampling=True, threshold=thresholds)

making dictionary :  3.024092197418213
unknown count :  4.03930139541626
unigram dictionary :  0.03992295265197754
subsampling time :  11.861744165420532


2. negative Sampling and Hierarchical Softmax
====

In [7]:
sample_table = Unigram_Table(vocab_size,sample_size,unigram_dict)
huffman_encode = Huffman_encoding(unigram_dict)
huffman_encode.encoding()
node_lst, node_dict, reverse_node_dict = huffman_encode.make_node_dict()

Filling Unigram Table


In [8]:
architectures = ['cbow']
use_loss = ['Negative_Sampling', 'Hierarchical_Softmax']
learning_rate = {'Softmax':1e-5, 'Negative_Sampling':0.05, 'Hierarchical_Softmax':0.07}

In [9]:
for architecture in architectures:
    for loss_type in use_loss:
        print ('Testing Model :%s, loss : %s' % (architecture, loss_type))
        model = Word2Vec(vocab_size, embedding_size, context_size, architecture=architecture, verbose=len(words_index)//50,
                        use_loss=loss_type, learning_rate = learning_rate[loss_type], sample_table=sample_table, encoder=node_dict,
                        decay_step=len(words_index)//20, decay_rate=0.98, neg_samples=sample_size, epochs=epochs)
        model.train(words_index)
        #print (learning_rate[loss_type])
        #learning_rate['Negative_Sampling'] = learning_rate[loss_type]
    # model save
np.save('./results/%d_%s_%s_input.npy' % (e, architecture, loss_type), model.params['inputVectors']) # 0.15
np.save('./results/%d_%s_%s_output.npy'% (e, architecture, loss_type), model.params['outputVectors']) # 0.15 

Testing Model :cbow, loss : Negative_Sampling
idx: 237581, loss: 17.995562, time: 47.821074, 
idx: 475162, loss: 17.790715, time: 50.704139, 
idx: 712743, loss: 17.514190, time: 50.414410, 
idx: 950324, loss: 17.412866, time: 49.125716, 
idx: 1187905, loss: 17.462446, time: 48.006757, 
idx: 1425486, loss: 17.451136, time: 47.782228, 
idx: 1663067, loss: 17.435149, time: 48.809634, 
idx: 1900648, loss: 17.285121, time: 49.540880, 
idx: 2138229, loss: 17.302104, time: 49.855042, 
idx: 2375810, loss: 17.254416, time: 48.343751, 
idx: 2613391, loss: 17.399910, time: 49.593298, 
idx: 2850972, loss: 17.311431, time: 49.009414, 
idx: 3088553, loss: 17.297997, time: 49.372493, 
idx: 3326134, loss: 17.278456, time: 49.265975, 
idx: 3563715, loss: 17.393832, time: 49.228044, 
idx: 3801296, loss: 17.429976, time: 49.352632, 
idx: 4038877, loss: 17.356720, time: 49.016695, 
idx: 4276458, loss: 17.397095, time: 49.564100, 
idx: 4514039, loss: 17.588610, time: 48.927411, 
idx: 4751620, loss: 17.6390

KeyboardInterrupt: 